In [1]:
import sys
import torch
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import time
import os
import numpy as np
import json
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import random
from tqdm import tqdm
import operator
import itertools
from scipy.io import  loadmat
import logging

from scipy import signal

from utils import data_transforms

from models.chong import ModelSpatial as GazeNet
from models.__init__ import save_checkpoint, resume_checkpoint
from dataloader.chong import GazeDataset, GooDataset
from dataloader import chong_imutils
from training.train_chong import train, test, GazeOptimizer

def generate_data_field(eye_point):
    """eye_point is (x, y) and between 0 and 1"""
    height, width = 224, 224
    x_grid = np.array(range(width)).reshape([1, width]).repeat(height, axis=0)
    y_grid = np.array(range(height)).reshape([height, 1]).repeat(width, axis=1)
    grid = np.stack((x_grid, y_grid)).astype(np.float32)

    x, y = eye_point
    x, y = x * width, y * height

    grid -= np.array([x, y]).reshape([2, 1, 1]).astype(np.float32)
    norm = np.sqrt(np.sum(grid ** 2, axis=0)).reshape([1, height, width])
    # avoid zero norm
    norm = np.maximum(norm, 0.1)
    grid /= norm
    return grid

def preprocess_image(image_path, eye):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # crop face
    x_c, y_c = eye
    x_0 = x_c - 0.15
    y_0 = y_c - 0.15
    x_1 = x_c + 0.15
    y_1 = y_c + 0.15
    if x_0 < 0:
        x_0 = 0
    if y_0 < 0:
        y_0 = 0
    if x_1 > 1:
        x_1 = 1
    if y_1 > 1:
        y_1 = 1

    h, w = image.shape[:2]
    face_image = image[int(y_0 * h):int(y_1 * h), int(x_0 * w):int(x_1 * w), :]
    # process face_image for face net
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
    face_image = Image.fromarray(face_image)
    face_image = data_transforms['test'](face_image)
    # process image for saliency net
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = data_transforms['test'](image)

    # generate gaze field
    gaze_field = generate_data_field(eye_point=eye)
    sample = {'image' : image,
              'face_image': face_image,
              'eye_position': torch.FloatTensor(eye),
              'gaze_field': torch.from_numpy(gaze_field)}

    return sample



def test_on_batch(net, dataloader, heatmap_size=64, save_dir='./sample_out/'):
    net.eval()
    heatmaps = []
    
    data = next(iter(dataloader))
    
    val_img, val_face, val_head_channel, eye_position, \
    val_gaze_heatmap, gaze, gaze_inside, image_paths, gt_box = data

    val_images = val_img.cuda()
    val_head = val_head_channel.cuda()
    val_faces = val_face.cuda()
    val_gaze_heatmap = val_gaze_heatmap.cuda()
    val_gaze_heatmap_pred, val_attmap, val_inout_pred = net(val_images, val_head, val_faces)
    val_gaze_heatmap_pred = val_gaze_heatmap_pred.squeeze(1)
    
    # Obtaining eval metrics
    final_output = val_gaze_heatmap_pred.cpu().data.numpy() 

    gaze_x, gaze_y = gaze
    gaze_x_cpu = gaze_x.cpu().data.numpy()
    gaze_y_cpu = gaze_y.cpu().data.numpy()
    gt_position = np.array([gaze_x_cpu, gaze_y_cpu]).T

    eye_x, eye_y = eye_position
    eye_x_cpu = eye_x.cpu().data.numpy()
    eye_y_cpu = eye_y.cpu().data.numpy()
    eye_position = np.array([eye_x_cpu, eye_y_cpu]).T
    
    gt_box = gt_box.numpy()

    N = val_img.shape[0]
    
    for idx in range(N):

        heatmap = final_output[idx].reshape([heatmap_size, heatmap_size])

        h_index, w_index = np.unravel_index(heatmap.argmax(), heatmap.shape)
        f_point = np.array([w_index / heatmap_size, h_index / heatmap_size])
        
        draw_result(image_paths[idx], eye_position[idx], heatmap, f_point, gt_position[idx],\
                    gt_box[idx], save_dir=save_dir, idx=idx)

def draw_result(image_path, eye, heatmap, gaze_point, gt_point, gt_box, save_dir, idx=0):
    
    im = cv2.imread(image_path)
    im_static = np.copy(im)
    
    x1, y1 = eye
    x2, y2 = gaze_point
    x3, y3 = gt_point
    
    image_height, image_width = im.shape[:2]
    x1, y1 = image_width * x1, y1 * image_height
    x2, y2 = image_width * x2, y2 * image_height
    x3, y3 = image_width * x3, y3 * image_height
    x1, y1, x2, y2, x3, y3 = map(int, [x1, y1, x2, y2, x3, y3])
    #cv2.circle(im, (x1, y1), 10, [255, 255, 255], -1)
    cv2.circle(im, (x2, y2), 15, [0, 0, 255], -1)
    cv2.circle(im, (x3, y3), 15, [0, 255, 0], -1)
    cv2.line(im, (x1, y1), (x2, y2), [0, 0, 255], 8)
    cv2.line(im, (x1, y1), (x3, y3), [0, 255, 0], 8)
    
    # Drawing the GT bounding box
    gt_box = gt_box * [image_width, image_height, image_width, image_height]
    gt_box = gt_box.astype(int)
    cv2.rectangle(im, (gt_box[0], gt_box[1]), (gt_box[2], gt_box[3]), (0,255,0), 6)

    # heatmap visualization
    heatmap = ((heatmap - heatmap.min()) / (heatmap.max() - heatmap.min()) * 255).astype(np.uint8)
    heatmap = cv2.resize(heatmap, (image_width, image_height))
    
    colormap = plt.get_cmap('gnuplot2')
    #colormap = plt.get_cmap('magma')
    heatmap = (colormap(heatmap)*255).astype(np.uint16)[:,:,:3]
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGB2BGR)

    heatmap = (0.75 * heatmap.astype(np.float32) + 0.25 * im_static.astype(np.float32)).astype(np.uint8)
    img = np.concatenate((im, heatmap), axis=1)   
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    filename = 'out_%s.png' % str(idx)
    save_path = save_dir + filename
    print(save_path)
    cv2.imwrite(save_path, img)

    return img


In [2]:
from utils_logging import setup_logger

# Logger will save the training and test errors to a .log file 
logger = setup_logger(name='first_logger',
                      log_dir ='./logs/',
                      log_file='modeltester.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

#Prepare dataloaders
test_images_dir = '/home/eee198/Documents/datasets/GOOReal/finalrealdatasetImgsV2/'
test_pickle_path = '/home/eee198/Documents/datasets/GOOReal/testrealhumans.pickle'
batch_size = 32

val_set = GooDataset(test_images_dir, test_pickle_path, 'test', use_gtbox=True)
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, num_workers=8, shuffle=False)

Number of Images: 2156


## With GOO-Synth Pretraining

In [3]:
# Load Model
net = GazeNet()
net.cuda()

# Resuming Training
resume_training = True
resume_path = './saved_models/chong_gooreal_final/model_5epochs_pretrain.tar'
if resume_training:
    net, _, _ = resume_checkpoint(net, None,resume_path)
    #test(net, test_data_loader,logger, save_output=True)


test_on_batch(net, test_data_loader, heatmap_size=64, save_dir='./sample_out_pretrained/')

=> loading checkpoint './saved_models/chong_gooreal_final/model_5epochs_pretrain.tar'
=> Optimizer has different parameter groups. Usually this will occur for staged optimizers (GazeNet, GazeMask)
=> loaded checkpoint './saved_models/chong_gooreal_final/model_5epochs_pretrain.tar' (epoch 420)
./sample_out_pretrained/out_0.png
./sample_out_pretrained/out_1.png
./sample_out_pretrained/out_2.png
./sample_out_pretrained/out_3.png
./sample_out_pretrained/out_4.png
./sample_out_pretrained/out_5.png
./sample_out_pretrained/out_6.png
./sample_out_pretrained/out_7.png
./sample_out_pretrained/out_8.png
./sample_out_pretrained/out_9.png
./sample_out_pretrained/out_10.png
./sample_out_pretrained/out_11.png
./sample_out_pretrained/out_12.png
./sample_out_pretrained/out_13.png
./sample_out_pretrained/out_14.png
./sample_out_pretrained/out_15.png
./sample_out_pretrained/out_16.png
./sample_out_pretrained/out_17.png
./sample_out_pretrained/out_18.png
./sample_out_pretrained/out_19.png
./sample_out_pre

## No Pretraining on GOO-Synth

In [4]:
# Load Model
net = GazeNet()
net.cuda()

# Resuming Training
resume_training = True
resume_path = './saved_models/chong_gooreal_final/model_5epochs_notrain.tar'
if resume_training:
    net, _, _ = resume_checkpoint(net, None,resume_path)
    #test(net, test_data_loader,logger, save_output=True)


test_on_batch(net, test_data_loader, heatmap_size=64, save_dir='./sample_out_nopretrain/')

=> loading checkpoint './saved_models/chong_gooreal_final/model_5epochs_notrain.tar'
=> Optimizer has different parameter groups. Usually this will occur for staged optimizers (GazeNet, GazeMask)
=> loaded checkpoint './saved_models/chong_gooreal_final/model_5epochs_notrain.tar' (epoch 25)
./sample_out_nopretrain/out_0.png
./sample_out_nopretrain/out_1.png
./sample_out_nopretrain/out_2.png
./sample_out_nopretrain/out_3.png
./sample_out_nopretrain/out_4.png
./sample_out_nopretrain/out_5.png
./sample_out_nopretrain/out_6.png
./sample_out_nopretrain/out_7.png
./sample_out_nopretrain/out_8.png
./sample_out_nopretrain/out_9.png
./sample_out_nopretrain/out_10.png
./sample_out_nopretrain/out_11.png
./sample_out_nopretrain/out_12.png
./sample_out_nopretrain/out_13.png
./sample_out_nopretrain/out_14.png
./sample_out_nopretrain/out_15.png
./sample_out_nopretrain/out_16.png
./sample_out_nopretrain/out_17.png
./sample_out_nopretrain/out_18.png
./sample_out_nopretrain/out_19.png
./sample_out_nopret